<h3>TODO</h3>
<p>Aquí nos encargamos de 2 cosas:</p>
<ol>
<li><a href='#CalculoPromedioPrecioUnitario'><b>Generar</b></a> la base de datos para calcular por regresión el promedio del precio unitario por barrio en el tiempo (<b style='color:green;'>OK</b>)
</li>
<li>Extraer los coeficientes del modelo (<b style='color:red;'>TODO</b>):
    <p>Se usará un refinamiento iterativo, en donde:</p>
        <ol>
            <li>Se parten de todos los coeficientes en 1</li>
            <li>Para cada columna i, se intentará seccionarlo en Xi partes
                <p>Por ejemplo, para distancias: de a 100 metros, de a 1 km, etc</p>
            </li>
            <li>Para cada sección Xi, de valor Vi, corresponde un coeficiente medio Ci, calculado de tomar el global y aislado de los demás coeficientes. En caso de no poseer ese dato, el registro no se toma en cuenta
                <p>Ejemplo:</p>
                    <ol>
                        <li>Sean las columnas C1, V1: (0, 10); C2, V2: (0, 10); C3, V3: (0, 1)</li>
                        <li>Para cada registro, se toma el coef global, se lo divide por el (C1;V1) y (C2;V2) correspondiente</li>
                        <li>Se reparten en los #V3 valores distintos y se toma el Cmedio de cada uno como los valores de C3</li>
                    </ol>
            </li>
            <li>Para calcular, se extrapolan los puntos de sección (Vi, Ci), de acuerdo al significado de la columna:
                <p>Si se trata de un valor binario, será valor V1 en ausencia y V2 en presencia</p>
                <p>Si se trata de valores discretos, podría ser una recta o una colección de puntos (valor, coef)</p>
                <p>Si se trata de una distancia, puede ser una recta, una cuadrática, exponencial, etc</p>
            </li>
        </ol>
</li>
</ol>

In [ ]:
# No vamos a graficar, de momento
#%matplotlib inline

import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
from os import listdir

<h3>Levantar DataFrame</h3>
<a name='LevantarDataFrame'/>

In [ ]:
def LeerDataFrame(nombreArchivo):
    """Función que lee un archivo y devuelve un dataframe"""
    return pd.read_csv(nombreArchivo, low_memory = False)

In [ ]:
def LeerTodosLosDataFrame(rutaCarpeta):
    """Lee todos los dataFrame y los concatena en uno solo"""
    listaDataFrames = []
    for archive in listdir(rutaCarpeta):
        if ".csv" in archive:
            print(archive)
            df = pd.read_csv(rutaCarpeta + archive, low_memory = False)
            listaDataFrames.append(df)
    return pd.DataFrame().append(listaDataFrames, ignore_index = True)

In [ ]:
import time
start = time.time()
gDataFrameGlobal = LeerTodosLosDataFrame('./processed/')
end = time.time()
totalTime = end - start
print('Time used: ' + str(totalTime))

<h3>DataFrame de precios promedios unitarios de los barrios en el tiempo</h3>
<a name = 'CalculoPromedioPrecioUnitario'/>

In [ ]:
def CalcularDataFramePromedioPrecioUnitario(dataFrame):
    """Genera el dataframe con los precios promedios unitarios de los barrios en el tiempo"""
    barriosPromedios = dataFrame.groupby(by = ['state_name', 'barrio', 'property_type', 'dump_date_year', 'dump_date_month'])['price_usd_per_m2'].mean()
    return barriosPromedios.reset_index(name = 'precio_unitario_promedio')

In [ ]:
gBarriosPrecioUnitario = CalcularDataFramePromedioPrecioUnitario(gDataFrameGlobal)

<h3>Cálculo de los coeficientes globales</h3>
<a name = 'CalculoCoeficienteGlobal'/>

In [ ]:
listaColumnas = ['state_name', 'barrio', 'property_type', 'dump_date_year', 'dump_date_month']
gDataFrameGlobal = pd.merge(gDataFrameGlobal, gBarriosPrecioUnitario, how = 'left', left_on = listaColumnas, right_on = listaColumnas)
gDataFrameGlobal['coeficiente_global'] = gDataFrameGlobal['price_usd_per_m2'] / gDataFrameGlobal['precio_unitario_promedio']

In [ ]:
gCoeficientes = [{'nombre': 'Cochera', 'metodo': 1, 'coeficiente': 1}]

gMetodos = [lambda x, y: y if x else 1]

def CalcularCoeficiente(valor, coeficiente):
    """Calcula un coeficiente"""
    if valor is None:
        return 1
    return gMetodos[coeficiente['metodo']](valor, coeficiente['coeficiente'])

def CalcularSerieCoeficiente(dataFrame, coeficiente):
    """Calcula una serie de coeficientes"""
    return dataFrame[coeficiente['nombre']].map(lambda x: CalcularCoeficiente(x, coeficiente))